<a href="https://colab.research.google.com/github/bithack07/TinyLLamaFineTune/blob/main/unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install transformers datasets trl

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-tl7cqu3e/unsloth_9820864d5cbe423fbd1e3e571333aa69
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-tl7cqu3e/unsloth_9820864d5cbe423fbd1e3e571333aa69
  Resolved https://github.com/unslothai/unsloth.git to commit 5ce83f272bd9e726b68b0a0452b8b04d32477133
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 13.4 MB/s eta 0:00:0

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import FastLanguageModel

In [7]:
# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-bnb-4bit",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
# Set the chat template for TinyLlama-1.1B-Chat-v1.0
tokenizer.chat_template = """{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n' + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"""

# Load your dataset
dataset = load_dataset("json", data_files="dataset.json", split="train")

In [21]:
# Define the formatting function to create the 'text' field
def format_example(example):
    messages = [
        {"role": "system", "content": example['system']},
        {"role": "user", "content": example['user']},
        {"role": "assistant", "content": example['assistant']}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": text}

In [22]:
# Map the dataset to add the 'text' field
dataset = dataset.map(format_example)

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [24]:
# Apply LoRA for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407
)

In [29]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    )
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/92 [00:00<?, ? examples/s]

In [30]:
# Start training
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 92 | Num Epochs = 6 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 25,231,360/4,000,000,000 (0.63% trained)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mayureshbhola8234 (mayureshbhola8234-abc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,7.723400
2,7.276400
3,7.705400
4,7.254800
5,6.523600
6,6.703500
7,6.072700
8,4.889000
9,5.315600
10,3.539700


TrainOutput(global_step=60, training_loss=1.2456480207877272, metrics={'train_runtime': 90.8222, 'train_samples_per_second': 5.285, 'train_steps_per_second': 0.661, 'total_flos': 201779447832576.0, 'train_loss': 1.2456480207877272})

In [31]:
# Save the model
model.save_pretrained("fine_tuned_tinyllama")
tokenizer.save_pretrained("fine_tuned_tinyllama")

('fine_tuned_tinyllama/tokenizer_config.json',
 'fine_tuned_tinyllama/special_tokens_map.json',
 'fine_tuned_tinyllama/tokenizer.model',
 'fine_tuned_tinyllama/added_tokens.json',
 'fine_tuned_tinyllama/tokenizer.json')